In [1]:
import numpy as np
import pandas as pd
import timeit


# list for pizza 'identifiers/numbers' and ingredients
def unda_data(file_lines):
    pizza_id = list()
    pizza_ingredients = list()
    for i in range(len(file_lines)-1):
        line_values = file_lines[i].split()

        if i == 0 : # Pick the number of pizzas ,and teams and create the dict 
            pizza_team_count_dict = {
                                    'no_of_pizzas': np.int(line_values[0] ) ,
                                    '2_member_teams': np.int(line_values[1] ) ,
                                    '3_member_teams': np.int(line_values[2] ) ,
                                    '4_member_teams': np.int(line_values[3])
                                    }
            number_of_teams = np.sum([int(i) for i in line_values[1:]]) # Get the total number of teams

        else : # Pick pizza types and ingredients 
            pizza_id.append(line_values[0])
            pizza_ingredients.append(line_values[1:])      

    # Get the total number of members 
    total_number_of_members = pizza_team_count_dict['2_member_teams']*2 + pizza_team_count_dict['3_member_teams']*3 + pizza_team_count_dict['4_member_teams']*4    
    # Create dataframe to store the pizza ingredient data         
    pizza_data = pd.DataFrame(columns=['no_of_ingredients','ingredients'])
    pizza_data['no_of_ingredients'] = pizza_id
    pizza_data['no_of_ingredients'] = pizza_data['no_of_ingredients'].astype(int) #to allow for math operations
    pizza_data['ingredients'] = pizza_ingredients
    # iterate over the one that is less which will now be the number of people served
    number_of_iterations =  min(pizza_team_count_dict['no_of_pizzas'] , total_number_of_members)
        
    
    return pizza_data ,pizza_team_count_dict,number_of_iterations

In [2]:
# reduce the number of teams with every iteration
def reduce_teams(team_size, dict_str_to_add):
    global team_sizes, pizza_team_count_dict
    key_string = str(team_size)+dict_str_to_add # Create the dictionary key to use for referencing
    pizza_team_count_dict[key_string] = pizza_team_count_dict[key_string] -1 
    if pizza_team_count_dict[key_string] == 0 :#remove team_size from team_sizes if there are no more teams of that size left
        team_sizes = [i for i in team_sizes if i != team_size]
    
    return 



#function to find combination that reduces the difference to 0
def team_sizes_for_zero_remainder(list_of_team_sizes, number_of_left_iterations): #list_of_team_sizes is empty initially
    global team_sizes 
    
    # from the remaining teams create a combination that will reduce the number of iterations to zero
        #since either all members get or none gets
    while number_of_left_iterations > 0 :
        for team_size in team_sizes:
            if team_size % 2 == number_of_left_iterations % 2  and  number_of_left_iterations>=team_size: #even vs odd
                number_of_left_iterations =  number_of_left_iterations - team_size
                list_of_team_sizes.append(team_size)
                reduce_teams(team_size,'_member_teams')
        
    return  list_of_team_sizes  

In [3]:
def individual_member(ingredients_list,selected_pizzas): #which pizza a crtain member/person gets
    global df
    selected_pizzas = str()
    
    row_with_max_variance_val = df['ingredients'].apply(lambda x : (len(list (set(x)-set(ingredients_list)) )) ).idxmax()

    #get the index of row with the max difference,,, the first row with max value is picked
     #If no pizza can increase variance, pick pizzas with with few values
    if all(i in df['ingredients'][row_with_max_variance_val] for i in ingredients_list):
        row_with_max_variance_val =  df['no_of_ingredients'].idxmin()

    #get a list of all ingredients so far
    ingredients_list.extend(list(set(df['ingredients'][row_with_max_variance_val]) - set(ingredients_list)))

    #Add the pizza id from the selected variance row
    selected_pizzas  = selected_pizzas+' '+str(row_with_max_variance_val) 

    #drop the selected row
    df = df.drop(row_with_max_variance_val)

    return selected_pizzas



#function to:
    # - deliver to a team of n members
    # - pick the value with the most variance/difference and that will be the next pizza 
    # - insert all selected pizzas for a single team into one result

def team_delivery(no_of_team_members):
    global submission_list,df,number_of_iterations

    if sum(team_sizes)<number_of_iterations:
        #list of ingredients
        ingredients_list = list()
        selected_pizzas = str()
        #get index of pizza with max ingredients
        row_with_max_ingredients = df['no_of_ingredients'].idxmax()
        #get a list of all ingredients so far
        ingredients_list.extend(list(set(df['ingredients'][row_with_max_ingredients]) - set(ingredients_list)))
        #cummulate pizza id
        selected_pizzas  = selected_pizzas+' '+str(row_with_max_ingredients) 
        #drop the selected row
        df = df.drop(row_with_max_ingredients)
        
        selected_pizzas_list = tuple(map(lambda n : individual_member(ingredients_list,selected_pizzas) ,np.arange(2,no_of_team_members+1)))
        
        number_of_iterations = number_of_iterations - no_of_team_members
        submission_string = str(no_of_team_members)+selected_pizzas+' '.join(selected_pizzas_list)
        submission_list.append(submission_string)
    
    return  

In [ ]:
start_time = timeit.default_timer()

path = 'e_many_teams.in' # 
file = open(path,'r') 
content = file.read()
#Split the created string into individual lines
file_lines = content.split('\n')

#prepare the data
pizza_data ,pizza_team_count_dict,number_of_iterations = unda_data(file_lines)
df = pizza_data.copy()
submission_list = list()
team_sizes = [4,3,2]

for i in range(number_of_iterations): #loop through all people that can be served
    #when the pizzas become few it checks whether they can fit 2 or 3 memeber teams instead of 4 
    #avoid having remainder 1.. the -1 caters for if we have a case like 10(avoid 1 remainder that causes an infinite loop)
    if sum(team_sizes)< number_of_iterations-1:
        list(map(lambda n: team_delivery(n),team_sizes)) 
        list(map(lambda n : reduce_teams( n ,'_member_teams' ),team_sizes))
                                       
    else:
        list_of_team_sizes = list()
        #get best combination that will return a zero
        list_of_team_sizes = team_sizes_for_zero_remainder(list_of_team_sizes,number_of_iterations)
        list(map(lambda n: team_delivery(n),list_of_team_sizes))
        break
        

file =  open('e submission file.out','w')
file.write(str(len(submission_list))) #number of teams served
file.write("\n")
for i in submission_list:
    file.write(i)
    file.write('\n')
file.write("\n\nTime it took : ")
file.write(str(timeit.default_timer() - start_time))
file.close()